# <span style="color: green; font-size: 40px; font-weight: bold;"> Projeto (Usando PySpark e Spark SQL)</span>

<br> <br>


# Analisando a Performance da Logística de Entrega com PySpark e Spark SQL

<br>

### Contexto

Neste mini-projeto, vamos explorar um contexto de negócios na área de logística: a **análise da performance de entregas de uma transportadora**. O projeto será desenvolvido desde a concepção do problema de negócio até a entrega de insights extraídos dos dados utilizando ferramentas comuns de análise de dados no dia a dia de um Cientista de Dados. Utilizaremos o Apache Spark SQL para realizar consultas e análises nos dados, com foco em agregação, funções Window e parse de data.

<br>

### Objetivo

O objetivo deste mini-projeto é **analisar um conjunto de dados de uma transportadora responsável por entregas de produtos**. Utilizaremos o Spark SQL para extrair insights e compreender como está a performance da logística de entrega da empresa. As soluções para cada pergunta de negócio serão apresentadas em Linguagem SQL e com o uso de funções do Spark SQL.

<br>

### Pergunta de Negócio Principal

> "Como podemos analisar a performance de entregas dos veículos de uma transportadora usando dados de horários de entregas?"

<br>

### Entregável

O entregável deste mini-projeto será uma **série de análises e insights sobre a performance de entregas de uma transportadora**. As análises serão realizadas utilizando o Spark SQL em um conjunto de dados fictício que simula os registros de entregas. O processo incluirá a concepção do problema de negócio, preparação dos dados e extração de insights.

<br>

### Sobre o Conjunto de Dados

Os dados utilizados neste mini-projeto são fictícios e representam os registros de entregas de uma transportadora. Cada registro contém o ID do veículo, a entrega realizada e o horário da entrega. O dataset é pequeno para permitir a realização rápida das consultas e demonstração de diversas funções e notações do Spark SQL.

<br>

O dataset possui 3 colunas:

<br>

<table border="2">
  <tr>
    <th style="text-align: center; font-size: 16px;">Nome da Coluna</th>
    <th style="text-align: center; font-size: 16px;">Tipo de Dado</th>
    <th style="text-align: center; font-size: 16px;">Descrição</th>
  </tr>
  <tr>
    <td>id_veiculo</td>
    <td>integer</td>
    <td>Identificação do veículo que realizou a entrega.</td>
  </tr>
  <tr>
    <td>entrega</td>
    <td>string</td>
    <td>Nome da entrega realizada.</td>
  </tr>
  <tr>
    <td>horario</td>
    <td>string</td>
    <td>Horário em que a entrega foi realizada (formato HH:MMa).</td>
  </tr>
</table>

<br> <br> <br>

# Importando Pacotes

In [6]:
# Imports

# Biblioteca principal do Apache Spark para processamento distribuído de grandes volumes de dados.
import pyspark

# Cria e gerencia a conexão com o cluster Spark.
from pyspark import SparkContext, SparkConf

# Ponto de entrada para criar DataFrames e utilizar funcionalidades do Spark SQL.
from pyspark.sql import SparkSession

# Cria especificações de janela para realizar operações como funções de ranking e agregação sobre partições 
# de dados.
from pyspark.sql import Window

# Refere-se a uma coluna em um DataFrame, permitindo manipulação de colunas.
from pyspark.sql.functions import col

# Atribui números únicos a linhas dentro de uma partição, com base na ordem especificada.
from pyspark.sql.functions import row_number

# Função de janela que retorna o valor da próxima linha em relação à linha atual dentro de uma partição.
from pyspark.sql.functions import lead  

# Funções agregadas que calculam o valor mínimo e máximo de uma coluna, respectivamente.
from pyspark.sql.functions import min, max

# Converte uma string de data/hora para um formato de timestamp Unix.
from pyspark.sql.functions import unix_timestamp

# Biblioteca para cálculos numéricos e manipulação de arrays
import numpy as np

<br> <br>

# <span style="color: green; font-size: 38px; font-weight: bold;">Preparando o Ambiente Spark</span>

In [11]:
# Definindo semente aleatória (seed) para reprodutibilidade do notebook
rnd_seed = 23
np.random.seed = rnd_seed
np.random.set_state = rnd_seed

# Se houver uma sessão Spark ativa, encerre-a
if 'sc' in globals():
    sc.stop()

if 'spark' in globals():
    spark.stop()


# Criando o Spark Context
conf = SparkConf().setAppName("Mini-Projeto4") \
                  .set("spark.ui.showConsoleProgress", "false") \
                  .set("spark.executor.heartbeatInterval", "20s") \
                  .set("spark.eventLog.enabled", "false") \
                  .set("spark.sql.shuffle.partitions", "2") \
                  .set("spark.sql.debug.maxToStringFields", "100") \
                  .set("spark.executor.memory", "4g") \
                  .set("spark.driver.memory", "4g") \
                  .set("spark.driver.maxResultSize", "2g")  # Configuração adicional para limitar o tamanho do resultado

# Criar o Spark Context e a Spark Session
sc = SparkContext(conf=conf)
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# Ajustar o nível de log para ERROR
sc.setLogLevel("ERROR")

# Configurar log4j para suprimir avisos (deixar como comentário e volta ao normal)
log4j_logger = sc._jvm.org.apache.log4j
log4j_logger.LogManager.getLogger("org").setLevel(log4j_logger.Level.ERROR)
log4j_logger.LogManager.getLogger("akka").setLevel(log4j_logger.Level.ERROR)

# Visualizar o objeto spark_session
spark

<br><br>

# <span style="color: green; font-size: 38px; font-weight: bold;">Carregando os Dados</span>

- Vamos carregar os Dados diretamente como **Dataframe do Spark** pois **não vamos fazer análise exploratória de dados**.

In [12]:
# Nome do arquivo
arquivo = 'projeto04/dados/dataset.txt'

# Carrega como dataframe do Spark
df = spark.read.csv(arquivo, header = True)

# Tipo
print(type(df))

# Visualiza primeiras 5 linhas
df.show(5)

<class 'pyspark.sql.dataframe.DataFrame'>
+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       298|Entrega 1|  7:58a|
|       298|Entrega 2|  8:04a|
|       298|Entrega 3|  8:17a|
|       298|Entrega 4|  8:28a|
|       298|Entrega 5|  8:33a|
+----------+---------+-------+
only showing top 5 rows



<br> <br>

# Criando Tabela Temporária

- Criamos uma tabela temporária para executar consultas SQL nos dados. A tabela temporária existe somente nesta sessão.

In [13]:
# Cria tabela temporária
df.createOrReplaceTempView("tb_logistica")

<br>

## Executando Queries SQL

- Vamos através de Queries visualizar os dados

In [14]:
# Verificando nome das colunas da tabela
spark.sql("SHOW COLUMNS FROM tb_logistica").show()

+----------+
|  col_name|
+----------+
|id_veiculo|
|   entrega|
|   horario|
+----------+



In [15]:
# Descreve a tabela para visualizar os tipos de dados das colunas
spark.sql("DESCRIBE TABLE tb_logistica").show()

+----------+---------+-------+
|  col_name|data_type|comment|
+----------+---------+-------+
|id_veiculo|   string|   NULL|
|   entrega|   string|   NULL|
|   horario|   string|   NULL|
+----------+---------+-------+



In [16]:
# Visualizando os 5 primeiros registros
spark.sql("SELECT * FROM tb_logistica LIMIT 5").show()

+----------+---------+-------+
|id_veiculo|  entrega|horario|
+----------+---------+-------+
|       298|Entrega 1|  7:58a|
|       298|Entrega 2|  8:04a|
|       298|Entrega 3|  8:17a|
|       298|Entrega 4|  8:28a|
|       298|Entrega 5|  8:33a|
+----------+---------+-------+



In [17]:
# Describe da tabela (retornará null por causa do tipo dos dados das colunas)
spark.sql("DESCRIBE tb_logistica").show()

+----------+---------+-------+
|  col_name|data_type|comment|
+----------+---------+-------+
|id_veiculo|   string|   NULL|
|   entrega|   string|   NULL|
|   horario|   string|   NULL|
+----------+---------+-------+



<br>

## Queries SQL x Dot Notation no Spark SQL